In [118]:
from itertools import islice
import fractions
from fractions import Fraction

In [119]:
# Knuth's algorithm to partition ns into m sets.
# https://codereview.stackexchange.com/questions/1526/finding-all-k-subset-partitions
def algorithm_u(ns, m):
    def visit(n, a):
        ps = [[] for i in range(m)]
        for j in range(n):
            ps[a[j + 1]].append(ns[j])
        return ps

    def f(mu, nu, sigma, n, a):
        if mu == 2:
            yield visit(n, a)
        else:
            for v in f(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v
        if nu == mu + 1:
            a[mu] = mu - 1
            yield visit(n, a)
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                yield visit(n, a)
        elif nu > mu + 1:
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = mu - 1
            else:
                a[mu] = mu - 1
            if (a[nu] + sigma) % 2 == 1:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v

    def b(mu, nu, sigma, n, a):
        if nu == mu + 1:
            while a[nu] < mu - 1:
                yield visit(n, a)
                a[nu] = a[nu] + 1
            yield visit(n, a)
            a[mu] = 0
        elif nu > mu + 1:
            if (a[nu] + sigma) % 2 == 1:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] < mu - 1:
                a[nu] = a[nu] + 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = 0
            else:
                a[mu] = 0
        if mu == 2:
            yield visit(n, a)
        else:
            for v in b(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v

    n = len(ns)
    a = [0] * (n + 1)
    for j in range(1, m + 1):
        a[n - m + j] = j - 1
    return f(m, n, 0, n, a)

In [120]:
def concise_range(ns):
    if not ns: return ""
    lower, *tail = sorted(ns)
    upper = lower
    ranges = []
    for n in tail:
        if n == upper + 1:
            upper = n
        else:
            ranges.append( (lower, upper) )
            lower = n
            upper = n
    ranges.append( (lower, upper) )
    intervals = []
    for lower, upper in ranges:
        if lower == upper:
            intervals.append(str(lower))
        else:
            intervals.append("{}-{}".format(lower, upper))
    return ",".join(intervals)
        

In [121]:
concise_range([1,2,4,5,17])

'1-2,4-5,17'

In [122]:
p_roll = { total: Fraction(6-abs(total-7), 36) for total in range(2,12+1)}

# the set of all possible rolls
roll_support = list(p_roll.keys())

p_roll

{2: Fraction(1, 36),
 3: Fraction(1, 18),
 4: Fraction(1, 12),
 5: Fraction(1, 9),
 6: Fraction(5, 36),
 7: Fraction(1, 6),
 8: Fraction(5, 36),
 9: Fraction(1, 9),
 10: Fraction(1, 12),
 11: Fraction(1, 18),
 12: Fraction(1, 36)}

In [123]:
class Craps:
    def __init__(self, craps=[2, 3, 12], naturals=[7, 11], out=7):
        self.craps = craps
        self.naturals = naturals
        self.out = out
    
    @property
    def internally_consistent(self):
        if set(self.craps).intersection(self.naturals):
            return False
        elif self.out not in set(self.craps).union(self.naturals):
            return False
        return True
    
    def p_win(self):
        return sum(
            p_roll[total] * self.p_come_out(total) 
            for total in roll_support)
        
    def p_come_out(self, total):
        if total in self.craps:
            return 0
        elif total in self.naturals:
            return 1
        else:
            return self.p_when_on_point(total)

        
    def p_when_on_point(self, point):
        p_out = p_roll[self.out]
        p_point = p_roll[point]
        return p_point / (p_out + p_point)
    
    def __str__(self):
        return "craps: {}  naturals: {}  out: {}".format(
            concise_range(self.craps),
            concise_range(self.naturals),
            self.out)
    
    @property
    def sort_key(self):
        complexity = len(self.craps) + len(self.naturals)
        return (complexity, self.out)
    
    def __lt__(self, other):
        return self.sort_key, other.sort_key 
        

In [127]:
# the original game
p = Craps().p_win()
p_original = p
p, float(p), 2*(0.5-float(p))

(Fraction(244, 495), 0.49292929292929294, 0.014141414141414121)

In [128]:
def generate_craps_variants():
    for craps, naturals, _ in algorithm_u(roll_support, 3):
        for out in craps + naturals:
            craps_variant = Craps(craps, naturals, out)
            if craps_variant.internally_consistent:

                yield craps_variant
 

In [129]:
sum( 1 for _ in generate_craps_variants() )

229858

In [130]:
def decorate_with_scores(craps_variants):
    one_half = Fraction(1, 2)
    for craps_variant in craps_variants:
        p = craps_variant.p_win() 
        score = (one_half - p) if p < one_half else 1 - p
        yield (score, p, craps_variant)
    

In [131]:
%time scored_variants = list(decorate_with_scores(generate_craps_variants()))


Wall time: 19.7 s


In [132]:
%time sorted_variants = sorted(scored_variants)

Wall time: 8.32 s


In [133]:
for score, p, craps_variant in sorted_variants[:20]:
    print("{} : {}  {:.4f}%".format(craps_variant, p, 100*float(p)))

craps: 2,8,10,12  naturals: 3,7  out: 10 : 5039/10080  49.9901%
craps: 2,6,10,12  naturals: 3,7  out: 10 : 5039/10080  49.9901%
craps: 2,4,8,12  naturals: 3,7  out: 4 : 5039/10080  49.9901%
craps: 2-3,7,12  naturals: 4,8  out: 4 : 5039/10080  49.9901%
craps: 2-3,7,12  naturals: 4,6  out: 4 : 5039/10080  49.9901%
craps: 2,4,6,12  naturals: 3,7  out: 4 : 5039/10080  49.9901%
craps: 2-3,6  naturals: 4,12  out: 4 : 5039/10080  49.9901%
craps: 2,4,7  naturals: 3,6,12  out: 4 : 5039/10080  49.9901%
craps: 2,4,7  naturals: 3,8,12  out: 4 : 5039/10080  49.9901%
craps: 2-3,8  naturals: 4,12  out: 4 : 5039/10080  49.9901%
craps: 2,7,10  naturals: 3,6,12  out: 10 : 5039/10080  49.9901%
craps: 2,7,10  naturals: 3,8,12  out: 10 : 5039/10080  49.9901%
craps: 2-3,6,11  naturals: 4,8,10  out: 8 : 3563/7128  49.9860%
craps: 2-3,6,11  naturals: 4,8,10  out: 6 : 3563/7128  49.9860%
craps: 2-3,8,11  naturals: 4,6,10  out: 6 : 3563/7128  49.9860%
craps: 2-3,8,11  naturals: 4,6,10  out: 8 : 3563/7128  49.98

In [134]:
sum( p == Fraction(5039, 10080) for _, p, _ in scored_variants)

12

In [135]:
sum( p > p_original and p < Fraction(1,2) for _, p, _ in scored_variants)/len(scored_variants)

0.01621000791793194

In [136]:
with open('all_craps_variants.txt', 'w') as outfile:
    for craps_variant in generate_craps_variants():
        p = craps_variant.p_win()    
        outfile.write("{} : {}  {:.4f}%\n".format(craps_variant, p, 100*float(p)))

In [137]:
import random

class CrapsSimulation:
    def __init__(self, craps=[2, 3, 12], naturals=[7, 11], out=7):
        self.craps = craps
        self.naturals = naturals
        self.out = out
    
    def reset(self):
        self.point = None
        self.roll_log = []
    
    def roll(self):
        roll = random.randint(1, 6) + random.randint(1, 6)
        self.roll_log.append(roll)
        return roll
    
    def play(self):
        self.reset()
        
        come_out_roll = self.roll()
        if come_out_roll in self.craps:
            return 0
        elif come_out_roll in self.naturals:
            return 1
        else:
            self.point = come_out_roll
            return self.play_on_point()
    
    def play_on_point(self):
        while True:
            roll = self.roll()
            if roll == self.out:
                return 0
            if roll == self.point:
                return 1
    
    

In [ ]:
n = int(1e8)

original_game = CrapsSimulation(craps=[2, 3, 12], naturals=[7, 11], out=7)
fairest_variant = CrapsSimulation(craps=[2, 8, 10, 12],  naturals=[3, 7],  out=10)

outcomes = []
for sim in [original_game, fairest_variant]:
    win_loss_record = [ sim.play() for _ in range(n) ]
    p_win = sum(win_loss_record) / len(win_loss_record)
    outcomes.append(p_win)

outcomes